In [4]:
import pandas as pd
file_path = 'TheCountofMonteCristo.txt'

In [5]:
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
print(len(text))

2616449


In [7]:
# see the different characters in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"$&'()*,-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]`abcdefghijklmnopqrstuvwxyz
81


In [8]:
# create a mapping for the encode and decode of the characters
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # take a string of integers, output a string

print(encode('Hello world'))
print(decode(encode('Hello world')))

[33, 59, 66, 66, 69, 1, 77, 69, 72, 66, 58]
Hello world


In [9]:
# now we encode the text and store it in a pytorch tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:1000])

torch.Size([2616449]) <built-in method type of Tensor object at 0x7652837c2810>
tensor([ 0, 28, 62, 55, 70, 74, 59, 72,  1, 14,  0,  0, 38, 55, 72, 73, 59, 63,
        66, 66, 59, 73,  1, 11, 11,  1, 45, 62, 59,  1, 26, 72, 72, 63, 76, 55,
        66,  0,  0, 40, 68,  1, 74, 62, 59,  1, 15, 17, 74, 62,  1, 69, 60,  1,
        31, 59, 56, 72, 75, 55, 72, 79, 10,  1, 14, 21, 14, 13, 10,  1, 74, 62,
        59,  1, 66, 69, 69, 65, 11, 69, 75, 74,  1, 55, 74,  1, 57, 69, 67, 70,
        69, 73, 59, 58,  1, 74, 62, 59,  1, 57, 72, 59, 77, 10,  1, 73, 70, 72,
        55, 68, 61,  1, 74, 69,  1, 74, 62, 59, 63, 72,  1, 72, 59, 73, 70, 59,
        57, 74, 63, 76, 59,  1, 73, 74, 55, 74, 63, 69, 68, 73,  1, 55, 74,  1,
        74, 62, 59,  0, 39, 69, 74, 72, 59, 11, 29, 55, 67, 59,  1, 58, 59,  1,
        66, 55,  1, 32, 55, 72, 58, 59,  1, 73, 63, 61, 68, 55, 66, 66, 59, 58,
         1, 74, 62, 59,  1, 74, 62, 72, 59, 59, 11, 67, 55, 73, 74, 59, 72, 10,
         1, 74, 62, 59,  1, 73, 70, 55, 

In [10]:
# separating the dataset into a train and a test dataset
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 8
train_data[:block_size + 1]

tensor([ 0, 28, 62, 55, 70, 74, 59, 72,  1])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target=y[t]
    print(f"when the input is {context} the target is: {target}")

when the input is tensor([0]) the target is: 28
when the input is tensor([ 0, 28]) the target is: 62
when the input is tensor([ 0, 28, 62]) the target is: 55
when the input is tensor([ 0, 28, 62, 55]) the target is: 70
when the input is tensor([ 0, 28, 62, 55, 70]) the target is: 74
when the input is tensor([ 0, 28, 62, 55, 70, 74]) the target is: 59
when the input is tensor([ 0, 28, 62, 55, 70, 74, 59]) the target is: 72
when the input is tensor([ 0, 28, 62, 55, 70, 74, 59, 72]) the target is: 1


In [29]:
torch.manual_seed(777)
batch_size = 4 # independent sequences
block_size = 8 # length of each sequence

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('outputs:')
print(yb.shape)
print(yb)

print('---')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target is: {target}")

inputs:
torch.Size([4, 8])
tensor([[73, 69, 67, 59, 74, 62, 63, 68],
        [59,  1, 67, 63, 68, 63, 73, 74],
        [62, 63, 67,  1, 54, 59, 78, 57],
        [73, 74,  1, 77, 63, 74, 62,  1]])
outputs:
torch.Size([4, 8])
tensor([[69, 67, 59, 74, 62, 63, 68, 61],
        [ 1, 67, 63, 68, 63, 73, 74, 59],
        [63, 67,  1, 54, 59, 78, 57, 59],
        [74,  1, 77, 63, 74, 62,  1, 72]])
---
when input is [73] the target is: 69
when input is [73, 69] the target is: 67
when input is [73, 69, 67] the target is: 59
when input is [73, 69, 67, 59] the target is: 74
when input is [73, 69, 67, 59, 74] the target is: 62
when input is [73, 69, 67, 59, 74, 62] the target is: 63
when input is [73, 69, 67, 59, 74, 62, 63] the target is: 68
when input is [73, 69, 67, 59, 74, 62, 63, 68] the target is: 61
when input is [59] the target is: 1
when input is [59, 1] the target is: 67
when input is [59, 1, 67] the target is: 63
when input is [59, 1, 67, 63] the target is: 68
when input is [59, 1, 67, 6

In [35]:
# setting up the bigram language model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))

torch.Size([32, 81])
tensor(5.3115, grad_fn=<NllLossBackward0>)

zO*;u?O*'LYnd0tY6y'"v
2(!xzo6732NU238A FoSg?l3AkKU]Y 7;L2ZlW
mg3Xu&'(CQ(C4v2nDfZO)EcLYQNEHiAPZ,v9f41 i2Z,*xtyFAG7`v(("JZN:UWbR*4P3M4x6rIZh!Yo"zS$k;awahC40QV3bJo),7hC-y!HgL$e-iX)4x!)Wa[z[OPG$P[-"p8A8ehNFADpjpdv
:hxY
TQFA`qhNc`C'OrkzD
3?55B8Q*(4s7)zV8.
rmvxm;Sz*r-lirCX'!DGZ`mQp!62Q,n5MJZy20uqxCnyNmh?5OBBRj]$bczRx`;y;mJLYjcXF[uXl*2QC4q(Mp(!(h[,vyaw] ABsFo"e-JAkz[-A!GJZ?]&tOUO)
EwDV8y'1DnSbe2ty4!((8r&


In [36]:
# we need to train the model to make it make sense

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [42]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5781688690185547


In [43]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


"I u in stritourerthedstealoce thitrteedit ande henttes, thrreve

mpenowde Lur gmeatoronowe oe as cul
pas t ntobeatouned
ced In'Thutin tousr sunt."
"`ngens, weng, mit l, Ad, ang.
we bluime 7ZRoshers maich,
pan
"
Antocrt t PYo it becucugouty; he e, s itharbonaven ast Itirchedaseras. -- thevise
e lary t we.
s il t? wi, p."sheanes oned coryon tharefrsilllie ne
"w, g g reres hiaghend cre


s The ce pt
